<a href="https://colab.research.google.com/github/katkamrakesh/covid-19/blob/master/instalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#@title Double click on this line in the code cell. { display-mode: "form" }

insta_tag = 'nasa' #@param {type:"string"}
number_of_posts = "100" #@param {type:"string"}
print('You insta tag', insta_tag)

You insta tag nasa


In [20]:
#@title Click on play to fetch the feed from instagram. { display-mode: "form" }

#!pip install PyQuery
import urllib
import requests
import json
import os
import re
import pyquery
import pandas as pd
import numpy as np
import math
import plotly
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import warnings
warnings.filterwarnings("ignore")

all_posts = []
post_cnt=1
post_comments = []
post_df = []
max_post_iter=int(number_of_posts)//12
follower_count = 0
dfs = []
dfc = []

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
  
##-------------------Instagram crawling starts-------------------##
def get_twelve_post(edges):   
  global all_posts
  global post_cnt
  global post_df
  global post_comments
  #12 post of this for loop
  try:
    for edge in edges:
        ps_info = {}
        ps_info['post_id'] = post_cnt
        if len(edge['node']['edge_media_to_caption']['edges']) > 0:
            ps_info['post_text'] = edge['node']['edge_media_to_caption']['edges'][0]['node']['text']
        else:
            ps_info['post_text'] = ''
        ps_info['post_coment_count'] = edge['node']['edge_media_to_comment']['count']
        ps_info['post_likes'] = edge['node']['edge_media_preview_like']['count']
        ps_info['post_time']=str(pd.to_datetime(edge['node']['taken_at_timestamp'],unit='s'))
        
        shortcode = edge['node']['shortcode']
        url_shortcode = 'https://www.instagram.com/p/'+shortcode+'/?__a=1'
        with urllib.request.urlopen(url_shortcode) as temp_u:
            js_data = json.loads(temp_u.read().decode())
        photo_in_post_cnt=0
        post = []
        for row in js_data['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges']:
            info = {}
            info['text'] = row['node']['text']
            info['username'] = row['node']['owner']['username']  
            post_comments.append(info)
        
        ps_info['comments'] = post_comments
        post_df.append(ps_info)
        if 'edge_sidecar_to_children' in js_data['graphql']['shortcode_media']:
            edges_shortcode = js_data['graphql']['shortcode_media']['edge_sidecar_to_children']['edges']
            #num of pic in single post of this for loop            
            k = 1
            for edge_s in edges_shortcode:
                fn = 'post_'+str(post_cnt)+'_'+str(k)
                k += 1
                if edge_s['node']['is_video'] and edge_s['node']['video_url'] != 'https://static.cdninstagram.com/rsrc.php/null.jpg':
                    display_url = edge_s['node']['video_url']
                elif edge_s['node']['display_url']:
                    display_url = edge_s['node']['display_url']
                post.append([display_url,fn])
                photo_in_post_cnt=photo_in_post_cnt+1  
        else:
            fn = 'post_'+str(post_cnt)+'_1'
            if js_data['graphql']['shortcode_media']['is_video'] and js_data['graphql']['shortcode_media']['video_url'] != 'https://static.cdninstagram.com/rsrc.php/null.jpg':
                display_url = js_data['graphql']['shortcode_media']['video_url']
            elif js_data['graphql']['shortcode_media']['display_url']:
                display_url = js_data['graphql']['shortcode_media']['display_url']
            post.append([display_url,fn])
        all_posts.append(post)
        post_cnt=post_cnt+1
  except Exception as e:
      print(e)
      return None

print("Fetching data for instagram tag: ",insta_tag)
url = 'https://www.instagram.com/' + insta_tag

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
}
html = get_web_html(url)
user_id = re.findall('profilePage_([0-9]+)', html, re.S)[0]
doc = pyquery.PyQuery(html)
items = doc('script[type="text/javascript"]').items()

for item in items:
    if item.text().strip().startswith('window._sharedData'):
        js_data = json.loads(item.text()[21:-1], encoding='utf-8')
        edges = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        get_twelve_post(edges)
        
for index in range(0,max_post_iter):
    if (flag):
        url_next = 'https://instagram.com/graphql/query/?query_id=17888483320059182&id='+user_id+'&first=12&after='+cursor
        with urllib.request.urlopen(url_next) as temp_u:
            js_data = json.loads(temp_u.read().decode())
        edges = js_data['data']['user']['edge_owner_to_timeline_media']['edges']
        cursor = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        flag = js_data['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        get_twelve_post(edges)
    else:
        break

Fetching data for instagram tag:  nasa


In [21]:
#@title Click on play to fetch top likes and comments. { display-mode: "form" }


monthDic = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
            7:'Jul',8:'Aug',9:'Sept',10:'Oct',11:'Nov',12:'Dec'}

def get_web_html(url):
  try:
      response = requests.get(url, headers=headers)
      if response.status_code == 200:
          return response.text
      else:    
          print('Wrong：', response.status_code)        
  except Exception as e:
      print(e)
      return None

def plot_line(x,y,color,name) :
    tracer = go.Scatter(x = x,y = y,mode = "lines",
                        marker = dict(color = color,
                                      line = dict(width =1)),
                       name = name)
    return tracer

def plot_line1(x,y,color,name,text) :
    tracer = go.Scatter(x = x,y = y,mode = "lines",hovertext = text,
                        marker = dict(color = color,
                                      line = dict(width =1)),
                       name = name)
    return tracer



def plot_bar(x,y,color,name) :
    tracer = go.Bar(x = x,y = y,text=y,textposition = 'auto',
                        marker = dict(color = color,
                                      line = dict(width =1)),
                       name = name)
    return tracer

def plot_layout(title,xLabel,yLabel) :
    layout = go.Layout(dict(title = title,
                            xaxis_title=xLabel,
                            yaxis_title=yLabel,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                            zerolinewidth=1,ticklen=5,gridwidth=2),
                        margin = dict(b = 100)
                       )
                  )
    return layout


def plot_likes_graph(dfs):
  configure_plotly_browser_state()
  print('in plot')
  trace1  = plot_line(dfs.Date,dfs.post_likes,'blue','Likes')
  layout  = plot_layout("Post Likes","Date","#Likes")
  figure  = go.Figure(data = [trace1],layout = layout)
  figure.show()
  #py.iplot(figure)
  #fig.show()
  #figure.write_html("fig1.png")
  

def plot_montly_post(dfs_month_post):
  configure_plotly_browser_state()
  trace1  = plot_line(dfs_month_post.mon_year,dfs_month_post.post_id,'blue','Comments')
  layout  = plot_layout("Number of Post Trend","Month-Year","#Post")
  figure  = go.Figure(data = [trace1],layout = layout)
  #figure.write_html("fig1.png")
  py.iplot(figure)

def plot_comments_likes(dfs_month_likes):
  configure_plotly_browser_state()
  trace1  = plot_line1(dfs_month_likes.mon_year,dfs_month_likes.comment_count,'blue','Comments',dfs_month_likes.comment_count)
  trace2  = plot_line1(dfs_month_likes.mon_year,(dfs_month_likes.post_likes/100),'orange','Likes',dfs_month_likes.post_likes)
  layout  = plot_layout("Post Comments Vs Likes","Month-Year","#Post")
  figure  = go.Figure(data = [trace1,trace2],layout = layout)
  py.iplot(figure)

def week_day_bar(weekly_post_cnt):
  configure_plotly_browser_state()
  trace1  = plot_line(weekly_post_cnt.weekday,weekly_post_cnt.post_id,'blue','Line Graph')
  trace2  = plot_bar(weekly_post_cnt.weekday,weekly_post_cnt.post_id,'skyblue','Bar Graph')
  layout  = plot_layout("Post By WeekDay","Weekday","#Post")
  figure  = go.Figure(data = [trace1,trace2],layout = layout)
  #figure.write_html("fig1.png")
  py.iplot(figure)

def week_day_pie(weekly_post_cnt):
  configure_plotly_browser_state()
  trace = go.Pie(labels=weekly_post_cnt.weekday, values=weekly_post_cnt.post_id)
  layout  = plot_layout("Post By WeekDay PieChart","Weekday","#Post")
  figure  = go.Figure(data = [trace],layout = layout)
  #figure.write_html("fig1.png")
  py.iplot(figure)

def user_top_comments(topCUsers):
  configure_plotly_browser_state()
  trace = go.Pie(labels=topCUsers.username, values=topCUsers.post_id)
  layout  = plot_layout("Top User for Commenting","Top Users","#Post")
  figure  = go.Figure(data = [trace],layout = layout)
  #figure.write_html("fig1.png")
  py.iplot(figure)

def post_analysis():
  global post_df
  global dfs
  global dfc
  dfs = []
  dfc = []
  for ps in post_df:
      dfs.append([ps['post_id'],ps['post_text'],ps['post_coment_count'],ps['post_likes'],ps['post_time']])
      for cm in ps['comments']:
          dfc.append([ps['post_id'],cm['text'],cm['username']])

  dfs = pd.DataFrame(dfs)
  dfc = pd.DataFrame(dfc)
  print(dfs.shape, dfc.shape)
  dfs.columns = ['post_id','post_text','comment_count','post_likes','post_time']
  dfc.columns = ['post_id','text','username']
  dfs['Date'] = pd.to_datetime(dfs['post_time'])

  dfs['year'] = pd.DatetimeIndex(dfs['Date']).year
  dfs['month'] = pd.DatetimeIndex(dfs['Date']).month
  dfs['Mon_Year'] = dfs['Date'].dt.strftime('%b-%Y')
  dfs['weekday'] = dfs['Date'].dt.day_name()
  dfs['weekday_num'] = dfs['Date'].dt.weekday

  total_comments = math.ceil(dfs['comment_count'].sum())
  avg_comment = math.ceil(dfs['comment_count'].mean())
  total_likes = math.ceil(dfs['post_likes'].sum())
  avg_likes = math.ceil(dfs['post_likes'].mean())

  print("Total Comments: ",total_comments)
  print("Average Comments: ",avg_comment)
  print("Total Likes: ",total_likes)
  print("Average Likes: ",avg_likes)

post_analysis()

(108, 5) (265896, 3)
Total Comments:  507955
Average Comments:  4704
Total Likes:  113046805
Average Likes:  1046730


In [24]:
 #@title Click on play to plot likes trend. { display-mode: "form" }
 
configure_plotly_browser_state()
plot_likes_graph(dfs)


in plot


In [25]:
#@title Click on play to plot posts by month trend. { display-mode: "form" }

configure_plotly_browser_state()
dfs_month_post = dfs.groupby(['year','month']).count()[['post_id']]
dfs_month_post.reset_index(inplace=True)
dfs_month_post['mon'] = dfs_month_post['month']
dfs_month_post = dfs_month_post.replace({"mon": monthDic})
dfs_month_post['mon_year'] = dfs_month_post['mon'] +'-'+dfs_month_post['year'].astype(str)
plot_montly_post(dfs_month_post)


In [27]:
#@title Click on play to plot likes and comments trend. { display-mode: "form" }

configure_plotly_browser_state()
dfs_month_likes = dfs.groupby(['year','month']).mean()[['post_likes','comment_count']]
dfs_month_likes.reset_index(inplace=True)
dfs_month_likes['mon'] = dfs_month_likes['month']
dfs_month_likes = dfs_month_likes.replace({"mon": monthDic})
dfs_month_likes['mon_year'] = dfs_month_likes['mon'] +'-'+dfs_month_likes['year'].astype(str)
plot_comments_likes(dfs_month_likes)


In [28]:
#@title Click on play to plot posts trend by weekly. { display-mode: "form" }

weekly_post_cnt = dfs.groupby(['weekday','weekday_num']).count()[['post_id']]
weekly_post_cnt.reset_index(inplace=True)
weekly_post_cnt = weekly_post_cnt.sort_values('weekday_num')
week_day_bar(weekly_post_cnt)
week_day_pie(weekly_post_cnt)


In [29]:
#@title Click on play to get post with most likes. { display-mode: "form" }
top_post_likes = dfs[['post_text','comment_count','post_likes','post_time']].sort_values('post_likes',ascending=False).head(10)
print("Top 10 posts with most likes:")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(top_post_likes)


Top 10 posts with most likes:
                                            post_text  comment_count  \
65  A HERO is home. 🌎⁣ ⁣\n\n#CongratsChristina! Af...          18780   
23  What cosmic wonder did @NASAHubble capture on ...          12808   
93  It’s a new day. It’s a new dawn.⁣⁣⁣\n⁣⁣⁣\nAstr...          10125   
16  Sometimes... there’s more than meets the eye. ...          14479   
35  Are you @nasahubble? Because you seem...reflec...          11607   
50  Exploring Hell. 🔥 Are you up for the challenge...           8610   
56  Pluto still has our heart 💙⁣\n⁣\n90 years ago ...          11039   
98  The majesty of @NASAHubble imagery… from your ...          10714   
31  You are the center of somebody’s universe.⁣ 🌌\...          12347   
37  So you think you're an #influencer? 🌚🤳🏾⁣\n⁣\nC...           8823   

    post_likes            post_time  
65     3142893  2020-02-06 20:34:30  
23     2190514  2020-03-29 19:55:29  
93     2129733  2019-12-28 16:01:15  
16     2120072  2020-04-0

In [30]:
#@title Click on play to get post with most comments. { display-mode: "form" }

top_post_comments = dfs[['post_text','comment_count','post_likes','post_time']].sort_values('comment_count',ascending=False).head(10)
print("Top 10 posts with most likes:")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(top_post_comments)

Top 10 posts with most likes:
                                             post_text  comment_count  \
65   A HERO is home. 🌎⁣ ⁣\n\n#CongratsChristina! Af...          18780   
16   Sometimes... there’s more than meets the eye. ...          14479   
47   🚨 Stop. Drop. And apply to #BeAnAstronaut!⁣ ⁣\...          13419   
23   What cosmic wonder did @NASAHubble capture on ...          12808   
31   You are the center of somebody’s universe.⁣ 🌌\...          12347   
35   Are you @nasahubble? Because you seem...reflec...          11607   
56   Pluto still has our heart 💙⁣\n⁣\n90 years ago ...          11039   
100  ⁣\n2024, we’re coming for you. 🌎🚀👨‍🚀👩‍🚀🌖⁣ ⁣⁣\n...          11029   
98   The majesty of @NASAHubble imagery… from your ...          10714   
93   It’s a new day. It’s a new dawn.⁣⁣⁣\n⁣⁣⁣\nAstr...          10125   

     post_likes            post_time  
65      3142893  2020-02-06 20:34:30  
16      2120072  2020-04-06 21:41:11  
47      1403840  2020-03-02 19:16:50  
23      21

In [31]:
#@title Click on play to get top users who commented the most. { display-mode: "form" }


topCUsers = dfc.groupby('username').count()
topCUsers.reset_index(inplace=True)
topCUsers = topCUsers[['username','post_id']].sort_values('post_id',ascending=False)[:15]
user_top_comments(topCUsers)